# 교차 검증과 그리드 서치
이런 저런 값으로 모델을 많이 만들어서 테스트 세트로 평가하면 결국 테스트 세트에 잘 맞는 모델이 만들어지지 않는가?  
훈련 세트에서 모델을 훈련하고 테스트 세트에서 모델을 평가.  
테스트 세트로 얻은 점수를 보고 실전에서 어느 정도 성능인지 기대. 다만, 테스트 세트를 사용해 자꾸 성능을 확인하면 점점 테스트 세트에 맞추게 되는 셈.  
따라서 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하면 안되고, 모델을 만들고 나서 마지막에 딱 한 번만 사용하는 것이 좋음.  
그렇다면 max_depth 매개변수를 사용해 하이퍼파라미터 튜닝을 어떻게 할 수 있을까요?

## 검증 세트
테스트 세트를 사용하지 않으면 과대/과소 적합을 판단하기 어려움.  
가장 간단히 테스트 세트를 사용하지 않고 이를 측정하는 방법은 훈련 세트를 또 나누면 된다.  
이 데이터를 **검증 세트**라고 한다.

전체 데이터 중 20%를 테스트 세트로, 나머지 훈련 세트 중에서 다시 20%를 떼어 검증 세트로 만들기.

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가.  
이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델 고르기.  
그다음 이 매개변수를 사용해 훈련 세트, 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련.  
그리고 마지막에 테스트 세트에서 최종 점수 평가.

판다스로 CSV 데이터 읽기

In [4]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

class 열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장.

In [5]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

훈련 세트와 테스트 세트 나누기

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

train_input과 train_target을 다시 train_test_split() 함수에 넣어  
훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target 만들기.  
여기서도 test_size 매개변수를 0.2로 지정하여 20% 비율 유지.

In [7]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

단순히 train_test_split() 함수를 2번 적용한 것임.  
  
훈련 세트와 검증 세트의 크기 확인.

In [8]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


이제 모델 만들고 평가

In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


이 모델은 확실히 훈련 세트에 과대 적합되어 있으므로 매개변수를 바꿔서 더 좋은 모델을 찾아야 함.

그 전에 검증 세트에 대해 더 알아야 할 것이 있음.

## 교차 검증
검증 세트를 만드느라 훈련 세트가 줄었고, 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어진다.  
그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것임.  
이럴 때 **교차 검증**을 이용.

교차 검증은 검증 세트를 뗴어 내어 평가하는 과정을 여러 번 반복, 그리고 이 점수를 평균하여 최종 검증 점수를 얻음.  

#### 3-폴드 교차 검증
훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것.

교차 검증의 경우 데이터의 80-~90%까지 훈련에 사용할 수 있고, 검증 세트가 줄지만 각 폴드에서 계산한 검증 점수를 평균하기에 안정된 점수로 생각할 수 있음.

사이킷런의 교차 검증 함수  
- cross_validate() : 평가할 객체를 첫 번째 매개변수로 전달, 검증 세트를 떼어내지 않고 훈련 세트 전체를 함수로 전달
- cross_val_score() 함수 : cross_validate() 함수의 결과 중에서 test_score 값만 반환

In [10]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01450133, 0.01502562, 0.01398063, 0.0139997 , 0.01597977]), 'score_time': array([0.00200725, 0.00199151, 0.0020206 , 0.00199938, 0.00200129]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환.  
처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미.  
각 키마다 5개의 숫자가 담겨 있는데 이는 이 함수가 기본적으로 5-폴드 교차 검증을 수행한다는 뜻이고,  
cv 매개변수에서 폴드 수를 바꿀 수 있다.

교차 검증 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻음.  
이름은 test_score지만 검증 폴드의 점수이므로 혼동하지 말 것.

In [11]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음.  
train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기에 따로 섞을 필요가 없었지만,  
교차 검증을 할 때는 훈련 세트를 섞어줘야 함. 섞으려면 분할기를 지정해주면 됨.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 줌.  
cross_validate() 함수는 회귀 모델일 경우 KFold 분할기를 사용하고  
분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StartifiedKFold를 사용.

In [12]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성.  
- n_splits 매개변수 사용

In [13]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부름.  
반면 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 함.  
사이킷런과 같은 머신러닝 라이브러리를 사용할 때 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현됨.

하이퍼파라미터를 튜닝하는 작업  
먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련.  
그 다음 검증 세트의 점수나 교차 검증을 통해 매개변수를 조금씩 바꿔봄.  
여러 매개변수를 바꿔가면서 모델을 훈련하고 교차 검증 수행.

다만, 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으려면 안됨. 독립적이지 않다는 소리.  
- 사이킷런의 GridSearchCV 클래스 사용 : 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행.

기본 매개변수를 사용한 결정 트리 모델에서 min_inpurity_decrease 매개변수의 최적값을 찾아보자.  
먼저 GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 릿그트를 딕셔너리로 만듬.

In [14]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듬.

In [15]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

그 다음 일반 모델을 훈련하는 것처럼 객체에 fit() 메서드 호출  
이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행  
그리고 cv 매개변수 기본값이 5여서 총 25개의 모델을 훈련한다고 보면 됨.  
많은 모델을 훈련하기에 병렬 실행에 필요한 CPU 코어 수를 지정하는 것이 좋아서 n_jobs 매개변수를 -1로 지정하여 모든 코어를 사용.

In [16]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서는 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하는데,  
그리드 서치는 훈련이 끝나면 25개 모델 중 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련. 이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있음.  
이 모델을 일반 결정 트리처럼 똑같이 사용할 수 있음.

In [17]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음.

In [18]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 편균 점수는 cv_results_ 속성의 mean_test_score 키에 저장되어 있음.

In [19]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


- argmax() 함수 사용 : 가장 큰 값의 인덱스를 추출할 수 있음. 그리고 이 인덱스를 사용해 params 키에 저장된 매개변수 출력 가능  
  
앞에서 출력한 gs_best_params_와 동일해야 함.

In [20]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


위 과정을 정리하면
1. 먼저 탐색할 매개변수 지정.
2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장됨.
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장됨.

결정 트리에서 min_inpurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정함. 여기에 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라 보면 아래와 같음.

In [21]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

- np.arange() 함수 : 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만듬.  
- 파이썬 range() 함수 : 비슷하지만 정수만 사용. max_depth를 5에서 20까지 1씩 증가하면서 15개 값을 만듬.
  min_samples_split은 2에서 100까지 10씩 증가하면서 10개의 값을 만듬.

따라서 이 매개변수로 수행하는 교차 검증 횟수는 9 * 15 * 10 = 1350개.  
기본 5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6750개나 됨.  
n_jobs=-1로 설정하고 그리드 서치 실행

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

최상의 매개변수 조합 확인

In [23]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검증 점수 확인

In [24]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


GridSearchCV 클래스를 사용하니 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개변수를 찾을 수 있다.  
그런데 앞에서 탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했는데 이렇게 간격을 둔 것에 특별한 근거가 없다.  
이보다 더 좁거나 넓은 간격으로 시도해 볼 수 있지 않나?

### 랜덤 서치
매개 변수 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 너무 만흥ㄴ 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있는데 이럴 때 **랜덤 서치**를 사용하면 좋다.

랜덤 서치에 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달.

*싸이파이*에서 2개의 확률 분포 클래스를 임포트  
싸이파이는 파이썬의 핵심 과학 라이브러리 중 하나. 적분, 보간, 선형 대수, 확률 등을 포함한 수치 게산 전용 라이브러리.

In [25]:
from scipy.stats import uniform, randint

uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는데 이를 균등 분호에서 샘플링한다고 말함.  
uniform은 실숫값, randint는 정숫값을 뽑음.  
0에서 10 사이의 범위를 갖는 randint 객체를 만들고 10개이 숫자를 샘플링.

In [26]:
rgen = randint(0, 10)
rgen.rvs(10)

array([6, 4, 6, 8, 9, 2, 1, 8, 8, 8], dtype=int64)

10개 밖에 되지 않아 고르게 샘플링되는 것 같지 않지만 샘플링 숫자를 늘리면 쉽게 확인 가능. 1000개를 샘플링해서 각 숫자의 개수를 세어보자.

In [27]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 90, 103, 104,  93,  99, 103,  84, 120, 106,  98], dtype=int64))

개수가 늘어나니 0에서 9까지의 숫자가 어느 정도 고르게 추출되었음.  
uniform 클래스도 같은 방법으로 사용하고, 0에서 1사이에서 10개를 추출하면 다음과 같다.

In [28]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.35991415, 0.49609411, 0.04394878, 0.70041483, 0.66485328,
       0.260708  , 0.85585219, 0.45975896, 0.10097474, 0.62750636])

난수 발생기랑 유사하게 생각하면 되고, 랜덤 서치에 randint, uniform 클래스를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있음. 샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋음.  
  
탐색할 매개변수의 딕셔너리 만들기.  
min_samples_leaf 매개변수를 탐색 대상에 추가. 이 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수.

In [29]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

min_imputiry_decrease는 0.0001에서 0.001 사이의 실숫값을 샘플링.  
max_depth는 20에서 50 사이의 정수.  
min_samples_split은 2에서 25 사이의 정수.  
min_samples_leaf는 1에서 25 사이의 정수를 샘플링.  
샘플링 횟수는 RandomizedSearchCV의 n_iter 매개변수에 지정.

In [30]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021B0B8B59C0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021B0BA7BCD0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021B0B8B5A50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021B0BA79D80>},
                   random_state=42)

params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾음.  
그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있음.  
  
최적의 매개변수 조합을 출력하면 다음과 같다.

In [31]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


최고의 교차 검증 점수는 다음과 같다.

In [32]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 이미 전체 훈련 세트(train_input, train_target)로 훈련되어 best_estimator_ 속성에 저장되어 있고,  
이 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인.

In [33]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


# 정리
레드 와인과 화이트 와인을 선별하는 작업의 성능을 끌어올리기 위해 결정 트리의 다양한 하이퍼파라미터를 시도해 봐야 함.  
이 과정에서 테스트 세트를 사용하면 결국 테스트 세트에 맞춰 모델을 훈련하는 효과를 만듬.  
테스트 세트는 최종 모델을 선택할 때까지 사용하지 말아야 한다.  
테스트 세트를 사용하지 않고 모델을 평가하려면 또 다른 세트가 필요한데 이를 검증(개발) 세트라고 부름.  
검증 세트는 훈련 세트 중 일부를 다시 덜어 내어 만듬.  
검증 세트가 크지 않으면 어떻게 데이터를 나누었는지에 따라 검증 점수가 들쭉날쭉할 것임.  
훈련한 모델의 성능을 안정적으로 평가하기 위해 검증 세트를 한 번 나누어 모델을 평가하는 것에 그치지 않고  
여러 번 반복할 수 있는데 이를 교차 검증이라고 한다.  
보통 훈련 세트를 5등분 혹은 10등분 하고 나누어진 한 덩어리를 폴드라고 부르며 한 폴드씩 돌아가면서 검증 세트의 역할을 한다.  
최종 검증 점수는 모든 폴드의 검증 점수를 평균하여 계산.  
교차 검증을 사용해 다양한 하이퍼파라미터를 탐색.  
머신러닝 라이브러리에서는 클래스와 메서드의 매개변수를 바꾸어 모델을 훈련하고 평가해보는 작업이다.  
이런 과정은 지루하고 반복적인데 테스트하고 싶은 매개변수 리스트를 만들어 이 과정을 자동화하는 그리드 서치를 사용하면 편리하다.  
매개변수 값이 수치형이고 특히 연속적인 실숫값이라면 싸이파이의 확률 분포 객체를 전달하여 특정 범위 내에서 지정된 횟수만큼 매개변수 후보 값을 샘플링하여 교차 검증을 시도할 수 있음.

## 확인문제

In [34]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021B0B8B59C0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021B0BA7BCD0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021B0B8B5A50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021B0BA79D80>},
                   random_state=42)

In [35]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
